<a href="https://colab.research.google.com/github/pstansfeld/MemProtMD/blob/main/MemProtMD_Insane.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/pstansfeld/MemProtMD/raw/main/mr-membrane-protein.png" height="200" align="right" style="height:240px">

##MemProtMD with Insane

[MemProtMD](https://doi.org/10.1016/j.str.2015.05.006) membrane protein insertion protocol adding a preformed lipid membrane around a protein structure. A database of [PDB](http://rcsb.org) structures can be found [here](http://memprotmd.bioch.ox.ac.uk).

The membrane-spanning regions and protein orientation is predicted using [Memembed](https://doi.org/10.1186/1471-2105-14-276); which may be downloaded from [here](https://github.com/psipred/MemEmbed).

The protein structure is converted to [Martini 3](https://www.nature.com/articles/s41592-021-01098-3) coarse-grained (CG) represenation using [Martinize](https://github.com/marrink-lab/vermouth-martinize) established in a lipid membrane using [Insane](https://doi.org/10.1021/acs.jctc.5b00209) and run for 10 ns of molecular dynamics simulations using [GROMACS](https://doi.org/10.1016/j.softx.2015.06.001).

The final snapshot of the simulation is converted back to [CHARMM36m](http://mackerell.umaryland.edu/charmm_ff.shtml#gromacs) representation using [CG2AT](https://doi.org/10.1021/acs.jctc.1c00295); which can be downloaded from [here](https://github.com/pstansfeld/cg2at).

Inspiration for this Google Colab was taken from the amazing work of the [ColabFold](https://github.com/sokrypton/ColabFold) team.

Change the settings below and click **Runtime → Run all**.

You will be prompted to either upload a Membrane Protein PDB file to simulate. Alternatively, if you select the RCSB you should enter a [PDB](http://rcsb.org) ID or if the files are from either [Swiss-Model](https://swissmodel.expasy.org/repository/) or [AlphaFold](https://alphafold.ebi.ac.uk) databases enter a UniProtID. You can also enter an amino acid sequence to model a TM Helix using [PyMOL](https://pymol.org/2/) or model the whole structure (up to 400 amino acids) using [ESMfold](https://esmatlas.com/resources?action=fold).

In [4]:
#@title Initialisation
%%capture
!pip install py3dmol
!apt-get update && apt-get upgrade -y && apt-get install -y gzip pymol dssp
Protein_Location = "Upload" #@param ["Upload","RCSB PDB","AFDB","Swiss-Model","Sequence"]
File_Location = Protein_Location

#@markdown ---
#@markdown #### Alpha Helical or Beta Barrel Membrane Protein?:
Secondary_Structure = "Alpha" #@param ["Alpha","Beta"]

#@markdown ---
MembraneType = "POPC 1" #@param ["POPC 1","POPE 4 POPG 1","POPE 7 POPG 2 CARD 1","Asymmetric","Other"]
#@markdown #### Options for other membrane types:
if MembraneType == "Other":
  OtherMembraneComposition = "" #@param {type:"string"}
  LowerMembraneComposition = OtherMembraneComposition
  UpperMembraneComposition = OtherMembraneComposition

#@markdown ---
#@markdown #### Options for asymmetric membranes:
elif MembraneType == "Asymmetric":
  UpperMembraneComposition = "POPC 9 DPG3 1" #@param {type:"string"}
  LowerMembraneComposition = "POPC 1" #@param {type:"string"}
else:
  LowerMembraneComposition = MembraneType
  UpperMembraneComposition = MembraneType

#@markdown ---
#@markdown #### Options for coordinates from databases:
#@markdown #### RCSB:
PDB_ID = "7SSU" #@param {type:"string"}
#@markdown #### AlphaFold or Swiss-Model database:
UniProtID = "P0A910" #@param {type:"string"}
#@markdown #### Amino Acid Sequence:
Sequence_name = 'Lgt' #@param {type:"string"}
Method = 'ESMfold' #@param ["ESMfold", "Helix"]
Sequence = 'MTSSYLHFPEFDPVIFSIGPVALHWYGLMYLVGFIFAMWLATRRANRPGSGWTKNEVENLLYAGFLGVFLGGRIGYVLFYNFPQFMADPLYLFRVWDGGMSFHGGLIGVIVVMIIFARRTKRSFFQVSDFIAPLIPFGLGAGRLGNFINGELWGRVDPNFPFAMLFPGSRTEDILLLQTNPQWQSIFDTYGVLPRHPSQLYELLLEGVVLFIILNLYIRKPRPMGAVSGLFLIGYGAFRIIVEFFRQPDAQFTGAWVQYISMGQILSIPMIVAGVIMMVWAYRRSPQQHVS' #@param {type:"string"}

In [ ]:
#@title Get PDB coordinate file
import os
import sys
import requests
import py3Dmol
from google.colab import files

sys.path.append('/usr/local/lib/python3.7/site-packages/')

os.chdir('/content/')

if File_Location == "Upload":
    upload = files.upload()
    filename = next(iter(upload))
elif File_Location == "RCSB PDB":
    name = str(PDB_ID.lower())
    os.system('wget http://www.rcsb.org/pdb/files/' + name + '.pdb1.gz')
    os.system('gunzip ' + name + '.pdb1.gz')
    filename = name + '.pdb'
    os.system('egrep -v "MODEL|ENDMDL" '+ name + '.pdb1 >'+ filename)
elif File_Location == "AFDB":
    name = str(UniProtID.upper())
    os.system(f'wget https://alphafold.ebi.ac.uk/files/AF-{name}-F1-model_v2.pdb')
    filename = name + '.pdb'
    os.rename(f'AF-{name}-F1-model_v2.pdb', filename)
elif File_Location == "Swiss-Model":
    name = str(UniProtID.upper())
    os.system(f'wget https://swissmodel.expasy.org/repository/uniprot/{name}.pdb')
    filename = name + '.pdb'
elif File_Location == "Sequence":
    name = Sequence_name
    if Method == "Helix":
        with open('helix.pml','w') as pml:
            pml.write(f'set retain_order,0\nset secondary_structure,1\nfor aa in "{Sequence}": cmd._alt(str.upper(aa))\nsave {Sequence_name}.pdb')
            !pymol -cq helix.pml
    elif Method == "ESMfold":
        esmfold_api_url = 'https://api.esmatlas.com/foldSequence/v1/pdb/'
        r = requests.post(esmfold_api_url, data=Sequence)
        structure = r.text
        with open(f"./{name}.pdb", 'w') as pdb_file:
            pdb_file.write(structure)
        filename = name + '.pdb'

def py3dmol_view(pdbfilename, working_dir):
    mview = py3Dmol.view(width=800, height=400)
    with open(working_dir + pdbfilename, "r") as f:
        mol1 = f.read()
    mview.addModel(mol1, "pdb")
    mview.setStyle({"cartoon": {"color": "spectrum"}})
    mview.setStyle({'resn': 'DUM'}, {'sphere': {}})
    mview.setStyle({'atom': 'BB'}, {'sphere': {}})
    mview.setStyle({'atom': 'PO4'}, {'sphere': {}})
    mview.setStyle({'atom': 'P'}, {'sphere': {}})
    mview.setStyle({'atom': 'PO2'}, {'sphere': {}})
    mview.setStyle({'atom': 'PO1'}, {'sphere': {}})
    mview.setBackgroundColor("0xffffff")
    mview.zoomTo()
    mview.show()

def energy_minimize(filename, working_dir):
    if not os.path.exists("temp"):
        os.makedirs("temp")
    os.chdir("temp")
    os.system(f"pdb2pqr30 --ff CHARMM --keep-chain {working_dir + filename} pqr.pdb")

    pdb2gmx(f="pqr.pdb", ignh=True, ff="charmm27", water="tip3p", o="conf.pdb")
    editconf(f="conf.pdb", d=8, c=True, o="conf.pdb")

    with open("em.mdp", "w") as em:
        em.write("integrator = steep\nnsteps = 5000\nemtol = 100\nemstep = 0.001")

    grompp(f="em.mdp", maxwarn=5, o="em", c="conf.pdb")
    mdrun(deffnm="em", c="clean.pdb")

    trjconv(f="clean.pdb", o=working_dir + f"fixed-{filename}", s="em.tpr", input=("system"))

    os.chdir("..")
    shutil.rmtree("temp", ignore_errors=True)

def run_martinize2():
    cmd = [
        "martinize2",
        "-f", "protein.pdb",
        "-ff", "martini3001",
        "-x", "protein-cg.pdb",
        "-o", "protein-cg.top",
        "-elastic",
        "-ef", "500",
        "-eu", "1.0",
        "-el", "0.5",
        "-ea", "0",
        "-ep", "0",
        "-merge", "A",
        "-maxwarn", "100000",
        "-scfix"
    ]
    subprocess.run(cmd, check=True)

def run_insane3(lipid, x, y, z, dm):
    lipid_args = lipid.split()
    cmd = [
        'python', '/content/insane3.py',
        *lipid_args, '-salt', '0.15', '-sol', 'W', '-o', 'CG-system.gro',
        '-p', 'topol.top', '-f', 'protein-cg.pdb', '-center',
        '-x', f"{x}", '-y', f"{y}", '-z', f"{z}", '-dm', f"{dm}",
    ]
    process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    if stdout:
        print(stdout.decode('utf-8'))
    if stderr:
        print(stderr.decode('utf-8'))

def update_topology_file():
    replacements = {
        'NA+': 'NA',
        'CL-': 'CL',
        '#include "martini_v3.itp"': (
            '#include "martini_v3.0.0.itp"\n'
            '#include "martini_v3.0.0_ions_v1.itp"\n'
            '#include "martini_v3.0.0_solvents_v1.itp"\n'
            '#include "martini_v3.0.0_phospholipids_v1.itp"\n'
        ),
    }
    lines = []
    with open('topol.top') as infile:
        for line in infile:
            for src, target in replacements.items():
                line = line.replace(src, target)
            lines.append(line)
    with open('topol.top', 'w') as outfile:
        for line in lines:
            outfile.write(line)

def process_membrane_composition(composition, flag):
    lipid = ""

    input_list = composition.split()

    for i in range(0, len(input_list), 2):
        lipid_type = input_list[i]
        quantity = input_list[i + 1]
        lipid += f"{flag} {lipid_type}:{quantity} "
        if lipid_type == 'KLA':
            lipid += "-au 1.2 "

    lipid = lipid.strip()
    return lipid

upper_lipid = process_membrane_composition(UpperMembraneComposition, '-u')
lower_lipid = process_membrane_composition(LowerMembraneComposition, '-l')
lipid = upper_lipid + ' ' + lower_lipid

name = os.path.splitext(filename)[0]
working_dir = '/content/' + name + '/'
os.makedirs(working_dir, exist_ok=True)
os.rename(filename, working_dir + name + '.pdb')
os.chdir(working_dir)

py3dmol_view(filename, working_dir)

In [ ]:
#@title Install dependencies
%%capture
os.chdir('/content/')
if not os.path.isdir("/content/memembed/"):
  !git clone https://github.com/timnugent/memembed
  %cd memembed
  !make
  %cd ../
  !pip install pdb2pqr vermouth gromacswrapper==0.8.3 MDAnalysis wget
  !git clone https://github.com/pstansfeld/cg2at
  !wget https://github.com/pstansfeld/MemProtMD/raw/main/martini_v300.zip
  !wget https://github.com/pstansfeld/MemProtMD/raw/main/insane3.py
  !wget https://github.com/pstansfeld/MemProtMD/raw/main/gromacs.zip
  !unzip -o martini_v300.zip
  !unzip -o gromacs.zip
  %mv /content/usr/local/gromacs/ /usr/local/
  !ln -s /usr/local/gromacs/bin/gmx /usr/bin/gmx

import gromacs
import MDAnalysis
import glob
import shutil
import subprocess
import wget
import pandas as pd
from gromacs import grompp, editconf, make_ndx, trjconv, confrms, pdb2gmx, mdrun

In [ ]:
#@title Energy Minimise & Fix Input Structure
%%capture
energy_minimize(filename, working_dir)

In [ ]:
#@title Predict Membrane Orientation
os.chdir(working_dir)
if Secondary_Structure == 'Alpha':
  os.system(f'/content/memembed/bin/memembed -o memembed.pdb fixed-{filename}')
elif Secondary_Structure == 'Beta':
  os.system(f'/content/memembed/bin/memembed -b -o memembed.pdb fixed-{filename}')

py3dmol_view('memembed.pdb', working_dir)

In [ ]:
#@title Set-up Coarse-Grained Membrane Protein System
%%capture
os.chdir(working_dir)

for file in glob.glob(r'/content/martini_v300/martini*.itp'):
    print(file)
    shutil.copy(file, working_dir)

make_ndx(f='memembed.pdb', o='index.ndx', input=('del 0', 'del 1-100','rDUM','q'),backup=False)
editconf(f='memembed.pdb',o='centered.pdb',n='index.ndx',c=True,d=3,input=(0,0),backup=False)

u = MDAnalysis.Universe('centered.pdb')

confrms(f2='memembed.pdb', f1='centered.pdb', one=True, o='aligned.gro', input=(3,3),backup=False)
editconf(f='aligned.gro', o='protein.pdb',label='A',resnr=1,n=True,input=(0,0),backup=False)

v = MDAnalysis.Universe('aligned.gro')
dum = v.select_atoms('resname DUM')
box = (u.dimensions/10.0)

dm = (box[2]/2) - ((dum.center_of_mass()[2]/10))

with open('protein.pdb', 'r') as file :
    filedata = file.read()
filedata = filedata.replace('HSE', 'HIS').replace('HSD', 'HIS').replace('MSE', 'MET').replace(' SE ', ' SD ')
with open('protein.pdb', 'w') as file:
    file.write(filedata)

run_martinize2()

with open('protein-cg.itp', 'w') as outfile:
    for file_name in glob.glob('molecule*.itp'):
        with open(file_name) as infile:
            for line in infile:
                if line.startswith('molecule'):
                    line = 'Protein 1\n'
                outfile.write(line)

run_insane3(lipid, round(box[0]), round(box[1]), round(box[2]), round(dm))

update_topology_file()

with open('em.mdp','w') as em:
            em.write('integrator = steep\nnsteps = 5000\nemtol = 1000\nemstep = 0.001')

grompp(f='em.mdp',o='em.tpr',c='CG-system.gro',maxwarn=10,backup=False,v=True)

mdrun(deffnm='em', c='CG-system.pdb',backup=False)

trjconv(f='CG-system.pdb', o='CG-system.pdb', pbc='res', s='em.tpr', conect=True, input='0',backup=False)

if not os.path.exists('MD'):
    os.mkdir('MD')

make_ndx(f='CG-system.pdb', o='index.ndx', input=('del 0', 'del 1-40', '0|rPOP*','1&!0','!1','del 1','name 1 Lipid','name 2 SOL_ION','q'),backup=False)

with open('cgmd.mdp','w') as md:
            md.write('integrator = md\ntinit = 0.0\ndt = 0.02\nnsteps = 10000\nnstxout = 0\nnstvout = 0\nnstfout = 0\nnstlog = 50000\nnstenergy = 50000\nnstxout-compressed = 50000\ncompressed-x-precision = 10000\nnstlist  = 10\nns_type  = grid\npbc   = xyz\ncoulombtype  = Reaction_field\nrcoulomb_switch = 0.0\nrcoulomb  = 1.1\nepsilon_r  = 15\nvdw_type  = cutoff\nrvdw_switch  = 0.9\nrvdw   = 1.1\ncutoff-scheme = verlet\ncoulomb-modifier = Potential-shift\nvdw-modifier  = Potential-shift\nepsilon_rf  = 0\nverlet-buffer-tolerance = 0.005\ntcoupl  = v-rescale\ntc-grps  = PROTEIN LIPID SOL_ION\ntau_t  = 1.0 1.0 1.0\nref_t  = 310 310 310\nPcoupl  = c-rescale\nPcoupltype  = semiisotropic\ntau_p  = 12.0\ncompressibility = 3e-4 3e-4\nref_p  = 1.0 1.0\ngen_vel  = yes\ngen_temp  = 310\ngen_seed  = -1\nconstraints  = none\nconstraint_algorithm = Lincs\ncontinuation  = no\nlincs_order  = 4\nlincs_warnangle = 30\n')

grompp(f='cgmd.mdp',o='MD/md',c='CG-system.pdb',maxwarn=10, n='index.ndx',backup=False)

In [ ]:
#@title Equilibrate Membrane Protein System
os.chdir(working_dir + 'MD')
mdrun(deffnm='md',backup=False, v=True, nsteps=10000, c='md.pdb')

py3dmol_view('md.pdb', f'{working_dir}MD/')

files_to_remove = [os.path.join(working_dir, f) for f in os.listdir(working_dir) if f.startswith("#")]
for file_path in files_to_remove:
    os.remove(file_path)


In [ ]:
#@title Convert back to Atomistic
os.chdir(working_dir)
shutil.rmtree('Atomistic', ignore_errors=True)

os.chdir(working_dir + 'MD/')
shutil.rmtree('CG2AT', ignore_errors=True)

# Convert CG to Atomistic
os.system('/content/cg2at/cg2at -group all -o align -w tip3p -c md.pdb -a ../fixed-' + name + '.pdb -loc CG2AT -ff charmm36-jul2020-updated -fg martini_3-0_charmm36')

os.chdir(os.path.join(working_dir, 'MD/CG2AT/FINAL/'))

# Run GROMACS commands
grompp(f='../../../em.mdp', o='final.tpr', c='final_cg2at_aligned.pdb', p='topol_final.top',maxwarn=10)

# Copy files
shutil.copytree(working_dir + 'MD/CG2AT/FINAL/', working_dir + 'Atomistic/')

os.chdir(os.path.join(working_dir, 'Atomistic'))
editconf(f='final.tpr',o=f"{name}-System.pdb")

# Rename files
os.rename(os.path.join(working_dir, 'MD', 'md.pdb'), os.path.join(working_dir, f"{name}-cgmd.pdb"))
os.rename(os.path.join(working_dir, 'MD', 'md.tpr'), os.path.join(working_dir, f"{name}-cgmd.tpr"))
os.rename(os.path.join(working_dir, 'memembed.pdb'), os.path.join(working_dir, f"{name}-memembed.pdb"))

# Residue numbering
p1 = MDAnalysis.Universe(f"../fixed-{name}.pdb")

p1residues = p1.select_atoms('name CA').resids
start = p1residues[0]

editconf(f=f"{name}-System.pdb",o=f"{name}-System.pdb",resnr=int(start),backup=False)
p2 = MDAnalysis.Universe(f"{name}-System.pdb")
p2residues = p2.select_atoms('name CA').resids

All = p2.atoms

df = pd.DataFrame({'original':p2residues.tolist(),'new':p1residues.tolist()})
df = df.set_index('original')

for i in All:
    if i.residue.resid in df.index:
        i.residue.resid = df.loc[i.residue.resid,'new']

All.write(f"{name}-System.pdb")

# Create index file
os.replace(os.path.join(working_dir, 'Atomistic', 'topol_final.top'),
           os.path.join(working_dir, 'Atomistic', 'topol.top'))
make_ndx(f=working_dir + 'Atomistic/' + name + '-System.pdb', o='index.ndx', input=('del 2-40', 'rSOL|rNA*|rCL*','1|2','0&!3','del 3','name 2 water_and_ions','name 3 Lipid','q'),backup=False)

# Edit topology file
input_file = working_dir + 'Atomistic/topol.top'
output_file = input_file

with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
    filedata = infile.read()
    filedata = filedata.replace('../FINAL/', '')
    outfile.write(filedata)

prod = 'https://raw.githubusercontent.com/pstansfeld/MemProtMD/main/mdp_files/500ns.mdp'
equil = 'https://raw.githubusercontent.com/pstansfeld/MemProtMD/main/mdp_files/10ns-pr.mdp'
filename = wget.download(prod)
filename = wget.download(equil)

In [ ]:
#@title View Atomistic System
py3dmol_view(f'Atomistic/{name}-System.pdb', working_dir)

In [ ]:
#@title Tidy up & Download
os.chdir(working_dir)
files_to_remove = [
    'em.trr',
    'em.tpr',
    'em.edr',
    'em.log',
    'protein.pdb',
    'molecule_0.itp',
    'chain_A.ssd',
    'mdout.mdp',
    'aligned.gro',
    'protein-cg.top'
]

for file in files_to_remove:
    try:
        os.remove(file)
    except OSError:
        pass

try:
    shutil.rmtree('MD')
except OSError:
    pass

# create directory and move files
os.makedirs('CG', exist_ok=True)
for file_name in os.listdir('.'):
    if os.path.isfile(file_name):
        shutil.move(file_name, 'CG')

# zip and download
zip_name = name + '.zip'
os.chdir('/content/')
shutil.make_archive(name, 'zip', name)
files.download(zip_name)